In [2]:
from ortools.linear_solver import pywraplp

In [94]:
def solve_integer_program(n, m, N, P, lambd, epsilon):
    # Create a solver
    solver = pywraplp.Solver.CreateSolver('CBC')  # 'CBC' is the solver used by ortools for integer programming
    #
    if not solver:
        return "Solver not available."
    #
    # Variables
    x = [solver.IntVar(0, 1, f'x{i}') for i in range(m)]  # Integer variables with lower bound 0 and upper bound 1
    zFP = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]  # Auxiliary variables z_i with lower bound 0 and upper bound 1
    zFN = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]
    zFN_pre = [solver.IntVar(0, 1, f'z{i}') for i in range(n)]
    y = [solver.NumVar(0, 1, f'z{i}') for i in range(n)]
    w = [solver.NumVar(0, 1, f'z{i}') for i in range(n)]
    
    #
    # Target function: 0^T x
    target_function = solver.Sum(x[i] * 0 for i in range(n))
    #
    # add constraints 
    for i in range(n):
        # Calculate N[i,:] * x (inner product)
        inner_product_N = solver.Sum(N[i][j] * x[j] for j in range(m))
        # Add constraint: z_i = min(N[i,:] * x, 1)
        solver.Add(inner_product_N <= y[i] + 1)
        solver.Add(inner_product_N <= y[i])
        solver.Add(zFP[i] <= inner_product_N)
        solver.Add(zFP[i] <= 1)
        solver.Add(zFP[i] >= 1 - (1 - y[i]))
        solver.Add(zFP[i] >= inner_product_N - y[i])
    
        #solver.Add(zFP[i] == min([inner_product_N, 1]))
        #
        inner_product_P = solver.Sum(P[i][j] * x[j] for j in range(m))
        #
        #solver.Add(zFN[i] == (1 - max([inner_product_P, 1])))
        solver.Add(inner_product_P <= w[i] + 1)
        solver.Add(inner_product_P <= (1-w[i]) + 1)
        solver.Add(zFN_pre[i] >= inner_product_P)
        solver.Add(zFN_pre[i] >= 1)
        solver.Add(zFN_pre[i] <= 1 + (1 - w[i]))
        solver.Add(zFN_pre[i] <= inner_product_P + w[i])
        solver.Add(zFN[i] == (1 - zFN_pre[i]))
        
            
        # Add constraint: sum(z_i) <= epsilon
        solver.Add(solver.Sum(zFP[i]/n + zFN[i]/n for i in range(n)) + lambd*solver.Sum(x[h] for h in range(m)) <= epsilon)
    
    
    # Objective function (to be minimized): 0 - 0^T x
    #solver.Minimize(target_function)
    
    
     # Solve the problem
    solutions = []

    while solver.NextSolution():
        solution = {}
        for i in range(m):
            solution[f'x{i}'] = x[i].solution_value()
        #for i in range(n):
        #    solution[f'z{i}'] = z[i].solution_value()
        solutions.append(solution)

    return solutions
    
    #
    # Solve the problem
    #status = solver.Solve()
    #
    #
    #
    #if status == pywraplp.Solver.OPTIMAL:
    #    # Print the results
    #    print("Status: Optimal")
    ##    print("Optimal Solution:")
    #    for i in range(m):
    #        print(f"x{i} = {x[i].solution_value()}")
    #    for i in range(n):
    #        print(f"z{i} = {z[i].solution_value()}")
    #else:
    #    print("The problem does not have an optimal solution.")

    


In [95]:
import numpy as np
from scipy.sparse import random

def generate_random_sparse_binary_matrix(rows, cols, density):
    # Generate a random binary matrix with the specified density (percentage of non-zero elements)
    random_matrix = random(rows, cols, density=density, format="csr", dtype=np.float64)
    random_matrix.data = np.array(random_matrix.data >= 0.5, dtype=np.int64)  # Threshold to get binary values
    return random_matrix

In [96]:
# Generate two random binary sparse matrices
rows = 5  # Number of rows in the matrix
cols = 6  # Number of columns in the matrix
density = 0.3  # Density of non-zero elements (percentage)

N = generate_random_sparse_binary_matrix(rows, cols, density)
P = generate_random_sparse_binary_matrix(rows, cols, density)

# Convert the sparse matrices to nested lists for OR-Tools
N = matrix1.toarray().tolist()
P = matrix2.toarray().tolist()


In [97]:
solutions = solve_integer_program(rows, cols, N, P, 0.1, 0.9)

In [98]:
solutions

[]